<a href="https://colab.research.google.com/github/sahil3Vedi/digit-recogniser-py/blob/master/Deep_Learning_on_MNIST_without_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
#declaring all the dependencies NO KERAS OR TENSORFLOW

import numpy as np
import csv
import math
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Neural Network is Trained in 4 Stages:

1) Network recieves input from model
FUNCTION: NN.getInput(model.inputList)
VARIABLES INVOLVED: NN.layers[0].inputVals

2) Network activates one layer after the other.
FUNCTION: NN.activate(activationFunction)
VARIABLES INVOLVED: NN.layers[i].inputVals, NN.layers[i].activations, NN.layers[i].activDerivates, NN.layers[i].layerWeights, NN.layers[i].bias, NN.layers[i].biasWeights

3) Model calculates cost from the target value of input data and answer provided by network.
FUNCTION: model.getCost(NN)
VARIABLES INVOLVED: NN.layers[-1].activations, NN.layers[-1].forwardError, model.cost

4) Network readjusts the weights based on the cost and error information.
FUNCTION: NN.backpropagate()
VARIABLES INVOLVED: NN.layers[i].forwardError, NN.layers[i].diffWeights, NN.layers[i].diffBias, NN.layers[i-1].activations, NN.layers[i-1].forwardError

In [0]:
class neuralNetwork:
  layers=[]
  finalError=[]
  inputElem=0
  hiddenElem=0
  outputElem=0
  
  #When a neural network is declared 
  def __init__(self):
    self.inputElem=784
    self.outputElem=10
    self.hiddenElem=50
    
    inputLayer = layer(self.inputElem,"normalize",None)
    self.layers.append(inputLayer)
    
    temp = layer(self.hiddenElem,"ReLU",self.layers[0])
    self.layers.append(temp)
    temp = layer(self.hiddenElem,"ReLU",self.layers[1])
    self.layers.append(temp)
    temp = layer(self.hiddenElem,"ReLU",self.layers[2])
    self.layers.append(temp)
    
    outputLayer = layer(self.outputElem,"softMax",self.layers[3])
    self.layers.append(outputLayer)
    #TOTAL 5 LAYERS
    
    print("Neural Network Initialised..")
    print("Number of layers: " + str(len(self.layers)))
    print("Size of Weight Mx at Input: " + str(np.shape(self.layers[0].layerWeights)))
    print("Size of Weight Mx at Hidden: " + str(np.shape(self.layers[1].layerWeights)))
    print("Size of Weight Mx at Hidden: " + str(np.shape(self.layers[2].layerWeights)))
    print("Size of Weight Mx at Hidden: " + str(np.shape(self.layers[3].layerWeights)))
    print("Size of Weight Mx at Output: " + str(np.shape(self.layers[4].layerWeights)))
    

In [0]:
class model:
  cost=0
  inputFile=[]
  trainingEntries=0
  
  def getInput(self,inputFile):
    with open(inputFile) as csvFile:
      csvReader = csv.reader(csvFile)
      self.inputFile = list(csvReader)
      self.trainingEntries = len(self.inputFile)-1  #n-1 because first row is header row
      print("Total entries excluding header: " + str(self.trainingEntries) + " rows")
    

In [0]:
class layer:
  numberElements=0
  bias=0
  activation="default"
  inputVals=[]
  activations=[]
  activDerivatives=[]
  forwardError=[]
  layerWeights=[]
  biasWeights=[]
  
  #When a layer is declared
  def __init__(self,no_of_elements,activation,previousLayer):
    self.numberElements=no_of_elements
    self.bias=0.5
    self.activation=activation
    #Initialising the Weight Matrix
    temp=[]
    if(previousLayer!=None):
      for i in range(previousLayer.numberElements):
        temptemp=[]
        for j in range(self.numberElements):
          temptemp.append(np.random.random_sample()-0.5)
        temp.append(temptemp)
    self.layerWeights=temp
    #Initialising the Bias Matrix
    for i in range(self.numberElements):
      self.biasWeights.append(np.random.random_sample()-0.5)
    

In [60]:
#-----MAIN FUNCTION-----

Model = model()
inputFile='drive/My Drive/MNIST Image CSV/train.csv'
Model.getInput(inputFile)

NN = neuralNetwork()
#Model.trainNetwork(NN)

Total entries excluding header: 42000 rows
Neural Network Initialised..
Number of layers: 5
Size of Weight Mx at Input: (0,)
Size of Weight Mx at Hidden: (784, 50)
Size of Weight Mx at Hidden: (50, 50)
Size of Weight Mx at Hidden: (50, 50)
Size of Weight Mx at Output: (50, 10)
